<a href="https://colab.research.google.com/github/Moukthika1253/NaiveBayesClassifier-Rotten-Tomato-reviews/blob/main/NBC_Rotten_tomato_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [10]:
df=pd.read_csv('/content/rt_reviews.csv',encoding='latin-1')
X=df.drop(columns=['Freshness']).copy()
y=df['Freshness']
X_train,X_remaining, y_train, y_remaining= train_test_split(X,y,train_size=0.5)
X_test, X_dev, y_test, y_dev = train_test_split(X_remaining,y_remaining,test_size=0.3)
print(f"X_train shape {X_train.shape}, y_train shape {y_train.shape}")
print(f"X_development shape {X_dev.shape}, y_development shape {y_dev.shape}")
print(f"X_test shape {X_test.shape}, y_test shape {y_test.shape}")


X_train shape (240000, 1), y_train shape (240000,)
X_development shape (72000, 1), y_development shape (72000,)
X_test shape (168000, 1), y_test shape (168000,)


In [3]:
categories=["Fresh","Rotten"]